In [101]:
!pip install torchkge
!pip install soynlp

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import cosine_similarity
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from torchkge.models import ConvKBModel
import pickle 
from copy import deepcopy
from tqdm import tqdm 
from kge_train import train_kge
from torchkge.utils.datasets import load_fb15k

import transformers
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein


torch.backends.cudnn.benchmark = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [103]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [104]:
HOME_DIR = "./"

In [105]:
#df = pd.read_csv("./datasets/20220920_csv/병원_triple.csv", encoding='cp949')
#df[530:550]

In [106]:
#df = pd.read_csv("./datasets/04 경찰서 병원 소방서/소방서_triple.csv", encoding='cp949')
FOLDER_NAME = "datasets/20220920_csv/"

In [107]:
path_to_file_folder = HOME_DIR+FOLDER_NAME
meta_data= ["file_name", "colum_name", "value_list"] #the meta-data should be extented based on EDA on the datasets
df_full = pd.DataFrame(columns=meta_data)
meta_data= ["head", "relation", "tail", "file_name"] #the meta-data should be extented based on EDA on the datasets
kg_full = pd.DataFrame(columns=meta_data)

for file_name in [file for file in os.listdir(path_to_file_folder)]:
    file_name = file_name
    a_file = HOME_DIR+FOLDER_NAME+file_name
    print("Reading the file: ", a_file)
    
    try:
        a_df = pd.DataFrame()
        if ".csv" in file_name:
            a_df = pd.read_csv(a_file, encoding='cp949', names=meta_data, header=None)
            a_df["file_name"] = file_name
        elif ".xlsx" in file_name:
            a_df = pd.read_excel(a_file, names=meta_data, header=None)
            a_df["file_name"] = file_name
        else:
            continue
        kg_full = kg_full.append(a_df, ignore_index=True)
        

        file_name_list = []
        colum_name_list = []
        value_list = []
        for col in a_df.columns:
            col_name = col
            col_values = list(a_df[col_name].values)
            file_name_list.append(file_name)
            colum_name_list.append(col_name)
            value_list.append(col_values)

        tmp_df = pd.DataFrame({"file_name": file_name_list,
                            "colum_name": colum_name_list,
                            "value_list": value_list})

        df_full = df_full.append(tmp_df, ignore_index=True)
        
    except:
        print("읽으려는 파일의 포멧을 확인해야함, 파일명: ", a_file,)


Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)07.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)06.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)02.csv
Reading the file:  ./datasets/20220920_csv/02 해운대_배수펌프(3075794).csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)05.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
읽으려는 파일의 포멧을 확인해야함, 파일명:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation).xlsx
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)04.csv
Reading the file:  ./datasets/20220920_csv/병원_triple.csv
Reading the file:  ./datasets/20220920_csv/ds_r_f15048634_202111221427(relation)03.csv
Reading the file:  ./datasets/20220920_csv/00 general_triple.csv
Reading the file:  ./datasets/20220920_csv/소방서_triple.csv
Reading the file:  ./datasets/20220920_csv/01 부산_유수

/tmp/ipykernel_1096315/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_1096315/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_1096315/4084913395.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kg_full = kg_full.append(a_df, ignore_index=True)
/tmp/ipykernel_1096315/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)
/tmp/ipykernel_1096315/4084913395.py:22: FutureWarning: The frame.append method is deprecate

Reading the file:  ./datasets/20220920_csv/경찰서_triple.csv


/tmp/ipykernel_1096315/4084913395.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df_full.append(tmp_df, ignore_index=True)


In [108]:
kg_full

,head,relation,tail,file_name
0,7633,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
1,7634,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
2,7635,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
3,7636,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
4,7637,start_time,0,ds_r_f15048634_202111221427(relation)07.csv
...,...,...,...,...
10438,광남지구대,latitude,35.14537016,경찰서_triple.csv
10439,망미2치안센터,latitude,35.1717912,경찰서_triple.csv
10440,부산사상경찰서,latitude,35.17452651,경찰서_triple.csv
10441,부산기장경찰서,latitude,35.32481808,경찰서_triple.csv


In [109]:
class Config:
    kge_n_epoch = 100
    kge_lr = 1e-4
    kge_batch = 64
    kge_margin = 0.5
    kge_conv_size = 3

In [110]:
config = Config()

In [111]:
#kg_full[["head", "relation", "tail"]].to_csv("triple_kb2.csv")
kg_full = kg_full.astype(str)

In [112]:
kb_emb_path = './kge_save'
os.makedirs(kb_emb_path, exist_ok=True)

In [113]:
#from torchkge.utils.datasets import load_fb15k
#kg, kg_val, kg_test = load_fb15k()
#
#model_convKB = ConvKBModel(256,
#                            3,
#                            kg.n_ent,
#                            kg.n_rel,
#)

In [114]:
kg, model_convKB = train_kge(kg_full[["head", "relation", "tail"]],
                             epochs=config.kge_n_epoch, 
                                lr=config.kge_lr, 
                                batch_size=config.kge_batch,
                                margin=config.kge_margin,
                                conv_size=config.kge_conv_size
                                )

originally counts 10443

user nunique 1250, item nunique 2457


Epoch 100 | mean loss: 4.53336: 100%|██████████| 100/100 [00:43<00:00,  2.30epoch/s]


In [115]:
emb_entity_ = model_convKB.get_embeddings()[0].detach().cpu().numpy()
emb_rel_ = model_convKB.get_embeddings()[1].detach().cpu().numpy()

ent_emb_dim = len(emb_entity_[0])
ent_vocab={}
ent_vocab['<unk>'] = 0
ent_vocab['<pad>'] = 1
ent_vocab.update({word: i+2 for word, i in kg.ent2ix.items()})
ent_ids = {i: word for word, i in ent_vocab.items()}

pad_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<pad>' token.
unk_emb_npa = np.zeros((1, ent_emb_dim))   #embedding for '<unk>' token.

ent_embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,emb_entity_))
print(ent_embs_npa.shape)
ent_emb = torch.nn.Embedding.from_pretrained(torch.from_numpy(ent_embs_npa).float(), freeze=False)

with open('ent_vocab_npa.npy','wb') as f:
    np.save(f,ent_vocab)

with open('ent_embs_npa.npy','wb') as f:
    np.save(f,ent_emb)


(3705, 256)


In [116]:
print(ent_emb(torch.LongTensor([10, 15]))) 
ids = ent_vocab["부곡지구대"]
print(ent_emb(torch.LongTensor([ids])))

tensor([[ 0.0152, -0.0386,  0.0253, -0.0403, -0.0233,  0.0279, -0.0197, -0.0037,
          0.0324,  0.0075, -0.0005,  0.0227, -0.0307,  0.0316,  0.0081,  0.0264,
          0.0059, -0.0098, -0.0427,  0.0028, -0.0144, -0.0316, -0.0042, -0.0256,
          0.0158, -0.0216, -0.0094,  0.0280,  0.0358, -0.0215,  0.0054,  0.0019,
         -0.0352,  0.0144, -0.0182,  0.0013,  0.0052,  0.0143, -0.0116,  0.0016,
          0.0157, -0.0124,  0.0270, -0.0380, -0.0354, -0.0186, -0.0138,  0.0279,
          0.0133,  0.0277, -0.0166,  0.0102,  0.0197, -0.0184,  0.0245,  0.0076,
          0.0275,  0.0104,  0.0377, -0.0113,  0.0063,  0.0006,  0.0107, -0.0072,
         -0.0145,  0.0313, -0.0250, -0.0399,  0.0067,  0.0254,  0.0384, -0.0303,
          0.0307,  0.0225,  0.0064,  0.0074, -0.0239,  0.0184, -0.0222, -0.0257,
         -0.0148, -0.0290, -0.0149,  0.0029, -0.0034, -0.0225,  0.0123,  0.0148,
          0.0065,  0.0193,  0.0393,  0.0054,  0.0181,  0.0266,  0.0288, -0.0172,
          0.0259,  0.0305,  

In [117]:
ent_name="부곡지구대"
e1_ids = ent_vocab[ent_name]
e1_torch = ent_emb(torch.LongTensor([e1_ids]))
e2_ids = np.arange(0, len(ent_vocab))
e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
similarity = cosine_similarity(e1_torch, e2_torch)
s_value, s_ids = torch.topk(similarity, 100)
s_ids = s_ids.squeeze()
s_ids

tensor([3018, 3241, 2982, 2905, 2997, 2906, 3689, 2928, 3374, 3318, 3412, 3279,
        3311, 3152, 2822, 3306, 2823, 3252, 2880,  535, 3463, 3307, 3377, 2995,
        2887, 3314, 3281, 3703, 3438, 3200, 3563, 3148, 3302, 2611, 3357, 2864,
        2923, 3633, 3565, 2865, 2006, 3174, 3151, 2978, 3171, 3215, 3217, 3160,
        3286, 3285, 3634, 2608, 3327, 3415, 3247, 3439, 3253, 3243, 3263, 2930,
        2337, 3414, 3159, 2927, 2924, 2860, 3150, 3129, 2979, 3016, 3156, 2993,
        3250, 2888, 2970, 3321, 2862, 3186, 1821, 2980, 3017, 2852, 3019, 3587,
         976, 3328, 3661, 3283, 3251, 3185, 3163, 3157, 3304, 2652, 2839, 3242,
        3701, 3165, 3657, 2821])

In [118]:
for idx in s_ids:
    print(idx)
    print(ent_ids[int(idx)])

tensor(3018)
부곡지구대
tensor(3241)
서면지구대
tensor(2982)
명장2동치안센터
tensor(2905)
남항해양파출소
tensor(2997)
반여3치안센터
tensor(2906)
내성지구대
tensor(3689)
행복한외과병원
tensor(2928)
대연치안센터
tensor(3374)
자성대파출소
tensor(3318)
용당119안전센터
tensor(3412)
좋은삼선병원
tensor(3279)
신호파출소
tensor(3311)
온천119안전센터
tensor(3152)
부산대학교병원
tensor(2822)
가덕도파출소
tensor(3306)
영선2치안센터
tensor(2823)
가야119안전센터
tensor(3252)
센텀119안전센터
tensor(2880)
금정소방서
tensor(535)
129.0759208
tensor(3463)
지사119안전센터
tensor(3307)
영선지구대
tensor(3377)
장안파출소
tensor(2995)
반여2
tensor(2887)
기장병원
tensor(3314)
왈레스기념침례병원
tensor(3281)
아미파출소
tensor(3703)
효성시티병원
tensor(3438)
중앙119안전센터
tensor(3200)
사직119안전센터
tensor(3563)
창선119안전센터
tensor(3148)
부산광역시의료원
tensor(3302)
영도병원
tensor(2611)
8470
tensor(3357)
우암파출소
tensor(2864)
구포부민병원
tensor(2923)
당감1치안센터
tensor(3633)
팔송파출소
tensor(3565)
청맥병원
tensor(2865)
구포성심병원
tensor(2006)
7902
tensor(3174)
부산진경찰서
tensor(3151)
부산남부경찰서
tensor(2978)
망미119안전센터
tensor(3171)
부산지방경찰청경찰특공대
tensor(3215)
산성산악119안전센터
tensor(3217)
삼육부산병원
tensor(3160)
부산북부소방서
tensor

In [119]:
def similarity_example_on_TransE(kg, ent_name="부곡지구대"):
    e1 = emb_entity_[kg.ent2ix[ent_name]]
    r1 = emb_entity_[kg.rel2ix['koad:roadNameCode']]
    e2 = emb_entity_[kg.head_idx]
    similarity = cosine_similarity(torch.Tensor([e1]), torch.Tensor(e2))
    s_value, s_ids = torch.topk(similarity, 10)
    for idx in s_ids:
        print(int(idx))
        print(kg.head_idx[int(idx)])
        print(kg.ix2ent[int(kg.head_idx[int(idx)])])

## text to KB entity module
 - input: text (entity name)
 - output: 

In [120]:
class KLDiveEmb(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim):
        super(KLDiveEmb, self).__init__()

        self.bert = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base')
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.drop_out = nn.Dropout(0.33)
        self.fc1 = nn.Linear(hidden_dim, out_dim)

    def forward(self, idx, mask):
    
        x = self.bert(idx, mask) 
        x = x.pooler_output
        x = F.relu(self.layer1(x))
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        
        return F.softmax(x, dim=1), x

In [121]:
class KLEDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data, max_token):
        
        self.tokenizer = tokenizer
        self.data = data
        self.max_token = max_token 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        e_name = self.data['e_name'][index] #Entity Name
        e_emb = self.data['e_emb'][index]  #KB embedding
            
        tokenized = self.tokenizer.encode_plus("".join(e_name),
                                                None,
                                                add_special_tokens=True,
                                                max_length = self.max_token,
                                                padding='max_length',
                                                truncation=True,
                                              )
        
        ids = tokenized['input_ids']
        mask = tokenized['attention_mask']

        return {'e_ids': torch.tensor(ids, dtype=torch.long), 
                'e_mask': torch.tensor(mask, dtype=torch.long),
                'target': torch.tensor(e_emb)}

In [122]:
model = KLDiveEmb(768, 512, 256).to(DEVICE)
criterion = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [123]:
k,i,v = [],[],[]
for ent, idx in ent_vocab.items():
    ent_name=ent
    e1_ids = ent_vocab[ent_name]
    e1_torch = ent_emb(torch.LongTensor([e1_ids]))
    k.append(ent)
    i.append(idx)
    v.append(e1_torch.squeeze().detach().numpy())

In [124]:
df = pd.DataFrame(columns=['e_name','e_idx', 'e_emb'])
df["e_name"] = k
df["e_idx"] = i
df["e_emb"] = v

In [125]:
train_df, valid_df = train_test_split(df, test_size=0.2)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
batch_size = 64

In [126]:
train_dataset = KLEDataset(tokenizer, train_df, max_token=20)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True, pin_memory=True)
valid_dataset = KLEDataset(tokenizer, valid_df, max_token=20) 
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size , num_workers=4, shuffle=False, pin_memory=True)

In [127]:
def train_fn(model, train_loader, optimizer, criterion):
    model.train()
    batch_loss = []
    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
        optimizer.zero_grad()
        e_ids = batch['e_ids'].to(DEVICE) #entity_name
        e_bert_mask = batch['e_mask'].to(DEVICE) 
        
        logits, _ = model(e_ids, e_bert_mask) 
                
        target = batch['target'].to(DEVICE) #KB embedding
        target = F.softmax(target, dim=1)
        target.requires_grad = False
        loss = criterion(logits.log(), target)

        batch_loss.append(float(loss.item()))
        loss.backward(loss)
        optimizer.step()
        
    return sum(batch_loss) / len(train_loader)

In [128]:
def valid_fn(model, valid_loader, criterion):
    model.eval()
    batch_loss = []
    with torch.no_grad():
        for idx, batch in tqdm(enumerate(valid_loader), total=len(valid_loader), leave=False): #학습 데이터를 batch size만큼씩 읽어옴
            optimizer.zero_grad()
            e_ids = batch['e_ids'].to(DEVICE) #entity_name
            e_bert_mask = batch['e_mask'].to(DEVICE) 
            
            logits, _ = model(e_ids, e_bert_mask) 
                    
            target = batch['target'].to(DEVICE) #KB embedding
            target = F.softmax(target, dim=1)
            loss = criterion(logits.log(), target)

            batch_loss.append(float(loss.item()))
        
    return sum(batch_loss) / len(valid_loader)

In [129]:
train_epoch_loss = []
test_epoch_loss = []
for epoch in range(50):
    train_avg_loss = train_fn(model, train_loader, optimizer, criterion)
    train_epoch_loss.append(train_avg_loss)
    print(train_avg_loss)

0.0011047788380466877


0.0007820807885319153


0.0007500254841362859


0.0007338873284096096


0.0007221453223734143


...

0.00042256106191532726


0.0004142943938124053


0.00041264049743520135


0.0004093153488901543


0.00039462706425841504


In [130]:
valid_avg_loss = valid_fn(model, valid_loader, criterion)
print(valid_avg_loss)

0.00031440665043191984


In [131]:
def prediction(model, keyword="춘천"):

    e_name = keyword
    tokenized = tokenizer.encode_plus("".join(e_name),
                                        None,
                                        add_special_tokens=True,
                                        max_length = 20,
                                        padding='max_length',
                                        truncation=True,
                                        )

    ids = tokenized['input_ids']
    mask = tokenized['attention_mask']

    batch = {'e_ids': torch.tensor(ids, dtype=torch.long), 
             'e_mask': torch.tensor(mask, dtype=torch.long)}

    e_ids = batch['e_ids'].to(DEVICE).unsqueeze(0) #entity_name
    e_bert_mask = batch['e_mask'].to(DEVICE).unsqueeze(0) 
    logits, word_emb = model(e_ids, e_bert_mask)
    e1_torch = word_emb.cpu()
    e2_ids = np.arange(0, len(ent_vocab))
    e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
    similarity = cosine_similarity(e1_torch, e2_torch)
    s_value, s_ids = torch.topk(similarity, 10)
    s_ids = s_ids.squeeze()
    s_ids

    return s_ids.detach().numpy(), s_value.detach().numpy()

In [132]:
def search_ex_matching(df, query = "<unk>"):
    keyword = query
    exist_boolean = (df["e_name"] == keyword).any()
    ex_matching = 0 #기본적으로 "<unk>"로 해둠
    if exist_boolean: 
        ex_matching = int(df[df["e_name"] == keyword]['e_idx']) 
    else: 
        ex_matching = 0

    return ex_matching

In [133]:
def levenshtein_sim(df, query):
    #idx = df["e_name"].apply(lambda x: levenshtein(x, query)).idxmin()
    idx = df["e_name"].apply(lambda x: jamo_levenshtein(x, query)).idxmin()
    return idx

In [134]:
def graph_ent_sim(ent_vocab, ent_emb, query, topN=5):
    ent_name=query
    e1_ids = ent_vocab[ent_name]
    e1_torch = ent_emb(torch.LongTensor([e1_ids]))
    e2_ids = np.arange(0, len(ent_vocab))
    e2_torch = ent_emb(torch.LongTensor([e2_ids])).squeeze()
    similarity = cosine_similarity(e1_torch, e2_torch)
    s_value, s_ids = torch.topk(similarity, topN)
    s_ids = s_ids.squeeze()
    return s_ids

In [163]:
query = "부산진병원"

#EXmatching 결과
ex_ids = search_ex_matching(df, query)
ex_sim_query = df[df['e_idx'] == int(ex_ids)]["e_name"].iloc[0]

#Levenshtein 유사도 결과
lev_ids = levenshtein_sim(df, query)
lev_sim_query = df[df['e_idx'] == int(lev_ids)]["e_name"].iloc[0]

print("---------- Graph similarity 유사도 결과 ----------")
if ex_ids != 0:
    graph_ids = graph_ent_sim(ent_vocab, ent_emb, ex_sim_query)
else:
    graph_ids = graph_ent_sim(ent_vocab, ent_emb, lev_sim_query)


for idx in graph_ids:
    print("유사 Entity: ", idx, " ", df.iloc[int(idx)]["e_name"])
print("---------- Graph similarity 유사도 끝 ----------")
print("")
print("")


#BERT-Entity 유사도 결과
print("---------- BERT-Entity 유사도 결과 시작 ----------")
s_ids, s_score = prediction(model, query)
total_ids = s_ids #np.concatenate([np.array([ex_ids]), np.array([lev_ids]), s_ids]) 
for seq_num, idx in enumerate(total_ids):
    print(df[df['e_idx'] == int(idx)][["e_idx","e_name"]].values.tolist())
print("---------- BERT-Entity 유사도 결과 끝 ----------")


---------- Graph similarity 유사도 결과 ----------
유사 Entity:  tensor(3158)   부산본병원
유사 Entity:  tensor(3241)   서면지구대
유사 Entity:  tensor(2822)   가덕도파출소
유사 Entity:  tensor(2928)   대연치안센터
유사 Entity:  tensor(3148)   부산광역시의료원
---------- Graph similarity 유사도 끝 ----------


---------- BERT-Entity 유사도 결과 시작 ----------
[[2569, '8428']]
[[3241, '서면지구대']]
[[535, '129.0759208']]
[[2286, '8156']]
[[1804, '7705']]
[[2996, '반여23파출소']]
[[2680, '8538']]
[[3215, '산성산악119안전센터']]
[[3691, '허브휴병원']]
[[2964, '동래병원']]
---------- BERT-Entity 유사도 결과 끝 ----------


In [164]:
most_sim_ent = df[df['e_idx'] == int(lev_ids)]["e_name"].iloc[0]

In [165]:
df[df['e_idx'] == int(lev_ids)]["e_idx"].iloc[0]

3158

In [166]:
sbj_possible_rel = kg_full[kg_full["head"]==most_sim_ent][["relation", "tail", "file_name"]]
obj_possible_rel = kg_full[kg_full["tail"]==most_sim_ent][["head", "relation", "file_name"]]

In [167]:
sbj_possible_rel

,relation,tail,file_name
5099,is_a,koad:hospital,병원_triple.csv
5207,kord:RoadNameAddress,부산 사하구 승학로 8,병원_triple.csv
5315,contact:phone,1599-8275,병원_triple.csv
5423,koad:roadNameCode,2638055000,병원_triple.csv
5531,longitude,128.9698294,병원_triple.csv
5639,latitude,35.10628188,병원_triple.csv


In [168]:
obj_possible_rel

,head,relation,file_name
